In [21]:
## Check optimal throughput IS achieved when dispatching 100% based on service time reduction/affinity
import sys
import os

module_path = os.path.abspath(os.path.join('../..'))
#print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
#print(sys.path)

from parallel import Invoker

# Calculate params for simulations
from numpy import arange,linspace,logspace
import math

NUM_SIMULATED_CORES = 16
STATIC_NUM_FUNCTIONS = 16
CACHE_SIZE=64
LIB_FRACTION = 0.25
F_STEP = 2
FunctionRange = [16]
workset_range = [40]
GroupingRange = logspace(base=2,start=0,stop=4,num=5)

def divisorGenerator(n):
    large_divisors = []
    for i in range(1, int(math.sqrt(n) + 1)):
        if n % i == 0:
            yield i
            if i*i != n:
                large_divisors.append(n / i)
    for divisor in reversed(large_divisors):
        yield divisor
            
def filter_configs_by_cores(num_funcs,grouping,divisor_list):
    num_queues = num_funcs / grouping
    if num_queues in divisor_list:
        return True
        return (num_funcs,grouping,cores_per_q)
    
# Cores_per_q is always even for now by virtue of filter_configs_by_cores
def setup_final_config(num_funcs,grouping,workset,affinity=False,ideal=False):
    num_queues = num_funcs / grouping
    cores_per_q = NUM_SIMULATED_CORES / num_queues
    return {'funcs': int(num_funcs), 'grouping' : int(grouping), 'workset' : workset,
            'cores_per_q': int(cores_per_q), 'affinity' : affinity, 'ideal' : ideal}
    
# Find best grouping model for a given working set, assuming BTB is contention
def best_grouping_btb(WorkingSet,core_divisors):
    unique_working_set = math.floor(WorkingSet * 1024 * (1-LIB_FRACTION))
    num_insts = unique_working_set/4
    num_branches = num_insts/5
    funcs_per_btb = math.floor(6*1024 / float(num_branches))
    
    largest_group = 1
    for x in core_divisors:
        if x > largest_group and x <= funcs_per_btb:
            largest_group = x
    
    return largest_group

# Find best grouping model for a given working set, assuming cache is contention
def best_grouping_cache(WorkingSet,core_divisors):
    unique_working_set = math.floor(WorkingSet * 1024 * (1-LIB_FRACTION))
    funcs_per_cache = math.floor((1024 * CACHE_SIZE) / float(unique_working_set))
    
    largest_group = 1
    for x in core_divisors:
        if x > largest_group and x <= funcs_per_cache:
            largest_group = x
    
    return largest_group


# Add theoretical best 99th% solution to list of params
# - 1 queue, 16 workers, ideal service time
#ideal_args = list(map(lambda x : setup_final_config(STATIC_NUM_FUNCTIONS,STATIC_NUM_FUNCTIONS,
#                                                    x,affinity=False,ideal=True),workset_range)) 
    # 0 working set = ideal serv. time
#print([ideal_args])
#final_params.extend(ideal_args)

# Make config for FIFO - 1x16
singleq_params = [setup_final_config(STATIC_NUM_FUNCTIONS,STATIC_NUM_FUNCTIONS,workset_range[0])]

# Make config for FIFO - (GF = best)
core_divisors = list(divisorGenerator(NUM_SIMULATED_CORES))
best_groupings = [(best_grouping_cache(w,core_divisors),w) for w in workset_range]
gf_params = list(map(lambda x : setup_final_config(STATIC_NUM_FUNCTIONS,x[0],x[1]),best_groupings))

# Make config for SST - 1x16
sst_params = [setup_final_config(STATIC_NUM_FUNCTIONS,STATIC_NUM_FUNCTIONS,workset_range[0],affinity=True)]
    
# Add affinity solution to parameter list 
#affinity_params = list(map(lambda x : setup_final_config(STATIC_NUM_FUNCTIONS,1,x,affinity=True),workset_range))
    # IS THIS RIGHT? shouldn't affinity be 1xN?
#final_params.extend(affinity_params)

In [22]:
### Function to setup a simulation param dict.
### Each runnable is in the directory 'exps'
def setup_sim(runnable_name,func_group,core_group,num_functions,load_range,workset,affinity,ideal):
    invokerArgs = {'runnableTarg' : runnable_name,
                  'mode': 'sweep_A',
                  'numProcs' : 24,
                  'NumWorkers' : NUM_SIMULATED_CORES,
                   'argrange': load_range,
                   'NumFunctions': num_functions,
                   'FunctionGrouping': func_group,
                   'CoreGrouping': core_group,
                   'WorkingSet' : workset*1024,
                   'RequestsToSimulate': 50000,
                  }
    if affinity is True:
        invokerArgs['UseAffinity'] = ''
    if ideal is True:
        invokerArgs['UseIdealSetup'] = ''
    return invokerArgs

from components.zipf_gen import ZipfKeyGenerator
from random import shuffle
import pandas

# Setup a load generator, a list of function popularities.
# Run the target simulation and return a pandas dataframe with all results
def run_sims(runnable_name,func_group,core_group,num_functions,load_range,workset,affinity,ideal):
    args = setup_sim(runnable_name,func_group,core_group,num_functions,load_range,workset,affinity,ideal)
    
    # Add to dictionary the function popularities assumed (use a zipf for now)
    zargs = { 'num_items' : num_functions, 'coeff' : 0} # 0 = uniform
    zgen = ZipfKeyGenerator(**zargs)
    pdf_array = [ zgen.prob_for_rank(i) for i in range(num_functions) ]
    shuffle(pdf_array)
    
    def make_string_from_pop_list(l):
        ostring = ""
        for x in l:
            ostring += str(x) + " "
        return ostring
    args['FuncPopularities'] = make_string_from_pop_list(pdf_array)
    simulationController = Invoker( **args )
    print('Starting simulation for (Nf,G)=(',num_functions,',',func_group,')')
    simulationController.startProcs()
    simulationController.joinProcs()
    # Get results
    numProcs = args['numProcs']
    
    results = [ simulationController.getResultsFromQueue(idx) for idx in range(numProcs) ]
    flat_results = [ y for x in results for y in x ]
    
    # Remap results to the following format to import to pandas
    # { 'loads' : [ list of data points ]
    #    <Percentile Value> : [ list of data points ]
    # }
    def add_key_or_append(d,k,v):
        if k in d.keys():
            d[k].append(v)
        else:
            d[k] = []
            
    remap_dict = {}
    
    for x in flat_results:
        for load,times in x.items():
            load_mrps = 1/(load*1e-9)/1e6
            remap_dict[load_mrps] = times
    sorted_dict = {k:v for k,v in sorted(remap_dict.items(),key=lambda x : x[0])}
    f_dict = {}
    
    for load,time_dict in sorted_dict.items():
        # Append all loads to dict
        add_key_or_append(f_dict,'load',load)
        for p,v in time_dict.items():
            add_key_or_append(f_dict,str(p),v)      

    return args,pandas.DataFrame.from_dict(f_dict)

from numpy import linspace,logspace
### Run a group of simulations for each pair of num_functions/function_groupings
load_range = logspace(start=2,stop=4,base=10,num=50)
#print(load_range)

### Setup simulations for static policies
###    FIFO -> uses the runnable target "qmodel_dispatch_nonuniform"
###    SST -> uses the target "qmodel_dispatch_sst"
gf_sims = list(map(lambda x : run_sims('qmodel_dispatch_nonuniform',x['grouping'],x['cores_per_q'],
                                        x['funcs'],load_range,x['workset'],x['affinity'],x['ideal']),gf_params))

singleq_sims = list(map(lambda x : run_sims('qmodel_dispatch_nonuniform',x['grouping'],x['cores_per_q'],
                                        x['funcs'],load_range,x['workset'],x['affinity'],x['ideal']),singleq_params))

Initializing harmonic sums...
Initializing pdf and cdf arrays....
Done!
Starting simulation for (Nf,G)=( 16 , 2 )
Simulation was unstable, last five service times from core 0 were: [100361.9378516458, 103932.69922378875, 107785.81008955813, 111542.59908625747] , killing sim.
Simulation was unstable, last five service times from core 0 were: [101151.87142280008, 102264.14755668129, 104124.99238325971, 104353.6880824676] , killing sim.
Simulation was unstable, last five service times from core 0 were: [101694.84633307617, 105417.32385451665, 109108.43878319956, 112271.03508289192] , killing sim.
Caught exception <Process(run) object at 0x7fd03d80bcf8> has terminated and cannot be interrupted. lets transparently ignore it
Simulation was unstable, last five service times from core 0 were: [100845.52215643533, 103575.66831596929, 106031.40679434789, 107412.69423925653] , killing sim.
Caught exception <Process(run) object at 0x7fd03d80ecc0> has terminated and cannot be interrupted. lets tran

Simulation was unstable, last five service times from core 0 were: [102852.79484419391, 106309.09716232918, 107025.42359385002, 110788.14735277876] , killing sim.
Simulation was unstable, last five service times from core 0 were: [102650.67406189376, 104584.70328806248, 106533.21268674653, 107516.19520407579] , killing sim.
Caught exception <Process(run) object at 0x7fd03d827550> has terminated and cannot be interrupted. lets transparently ignore it
Simulation was unstable, last five service times from core 0 were: [100258.65123454086, 103459.61003964859, 106772.80141135717, 107031.37641039988] , killing sim.
Caught exception <Process(run) object at 0x7fd03d827550> has terminated and cannot be interrupted. lets transparently ignore it
Simulation was unstable, last five service times from core 0 were: [100777.93047063201, 104318.4470548257, 105701.6869659042, 108861.5497408377] , killing sim.
Caught exception <Process(run) object at 0x7fd03d827550> has terminated and cannot be interrupt

Caught exception <Process(run) object at 0x7fd03a08a860> has terminated and cannot be interrupted. lets transparently ignore it
Simulation was unstable, last five service times from core 0 were: [101601.71562035338, 105536.05587968108, 107414.01362485027, 110006.2476753053] , killing sim.
Caught exception <Process(run) object at 0x7fd03a08a860> has terminated and cannot be interrupted. lets transparently ignore it
Simulation was unstable, last five service times from core 0 were: [104049.96918704458, 106916.21980400618, 109761.18652112558, 113475.82030071417] , killing sim.
Caught exception <Process(run) object at 0x7fd03a08a860> has terminated and cannot be interrupted. lets transparently ignore it
Initializing harmonic sums...
Initializing pdf and cdf arrays....
Done!
Starting simulation for (Nf,G)=( 16 , 16 )
Simulation was unstable, last five service times from core 0 were: [101439.78276454523, 104300.12068469988, 106971.93938675868, 110222.76360124521] , killing sim.
Simulation wa